# 02_00_CCD_Photometry

## 모듈 버전

photoutils는 1.12 버전을 사용해야 한다.

In [ ]:
!pip install photutils==1.12

## 설치된 모듈 확인

아래의 셀은 설치된 모듈을 버전까지 쉽게 확인할 수 있도록 코딩한 것입니다.

In [ ]:
import importlib, sys, subprocess
packages = "numpy, matplotlib, astropy, ccdproc, jupyter-book, pyppeteer, opencv-python, sep, astro_ndslice, seaborn, photutils, ysfitsutilpy, ysphotutilpy, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q']) # pip install
        #subprocess.check_call([sys.executable, '-m', 'conda', 'install', pkg, ]) # conda install
        print(f"**** {pkg} module is now being installed.")
    else:
        print(f"******** {pkg} module is already installed.")
print("#"*60)
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

2025년 1월 12일 현재 정상 동작하는 모듈 버전
* 0 Python     3.11.11 64bit [GCC 11.2.0]
* 1 IPython    8.27.0
* 2 OS         Linux 6.8.0 51 generic x86_64 with glibc2.39
* 3 numpy      1.26.4
* 4 matplotlib 3.9.2
* 5 astropy    6.1.3
* 6 ccdproc    2.4.2
* 7 jupyter-book 1.0.3
* 8 pyppeteer  1.0.2
* 9 opencv-python 4.10.0.84
* 10 sep        1.2.1
* 11 astro_ndslice 0.2
* 12 seaborn    0.13.2
* 13 photutils  1.12.0
* 14 ysfitsutilpy 0.2
* 15 ysphotutilpy 0.1.1
* 16 version_information 1.0.4

## import modules

In [2]:
from glob import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import EarthLocation

import _astro_utilities
import _Python_utilities

plt.rcParams.update({'figure.max_open_warning': 0})

import warnings
warnings.filterwarnings('ignore')

## 변수 설정

In [ ]:
count_stars = False
verbose = False
tryagain = False
trynightsky = False
tryASTROMETRYNET = True
# file_age = 365
# downsample = 4

## 프로젝트 폴더 

그리고 프로젝트 마다 폴더를 구분하여 저장하고 있으며, 프로젝트 폴더마다 전처리를 동일한 파일로 수행하는 방법을 택하고 있습니다. 
전처리 파일은 "CAL-BDF" 폴더에 모두 넣어두고 마스터 파일을 만들어서 사용합니다. 이때 폴더명/파일명은 pathlib을 이용하여 처리합니다.

In [3]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C2-Asteroid"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C3-EXO"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-09_-_RiLA600_ASI6200MMPro_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2024-11_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-11_-_RiLA600_ASI6200MMPro_-_3bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

# PROJECDIR = BASEDIR / "C5-Test"
# TODODIR = PROJECDIR / "-_-_-_-_GSON300_STF-8300M_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
if verbose == True :
    print ("DOINGDIRs: ", format(DOINGDIRs))
    print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    BDFDIR = Path(BDFDIR[0])    
except : 
    BDFDIR = TODODIR
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

filter_str = '2025-01-09'
DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
if verbose == True :
    print ("DOINGDIRs: ", DOINGDIRs)
    print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

## 환경 변수 설정

In [ ]:
#####################################################################
# Observed location
LOCATION = dict(lon=127.005, lat=37.308889, elevation=101)
Suwon =  EarthLocation(lon=127.005 * u.deg, 
                                 lat=37.308889 * u.deg, 
                                 height=101 * u.m)
observatory_code = "P64"

# Used for any `astropy.SkyCoord` object:
SKYC_KW = dict(unit=u.deg, frame='icrs')
#######################################################
# Initial guess of FWHM in pixel
FWHM_INIT = 4

# Photometry parameters
# R_AP = 1.5*FWHM_INIT # Aperture radius
# R_IN = 4*FWHM_INIT   # Inner radius of annulus
# R_OUT = 6*FWHM_INIT  # Outer radius of annulus

Mag_Low = 11.5
Mag_High = 15

Mag_target = 12.5
Mag_delta = 2
ERR_Max = 0.5

coord_deltas = np.arange(0.00001, 0.00050, 0.00001)
#######################################################

## Combine master BIAS, DARK, FLAT

하나의 ``TODODIR`` 안의 모든 관측 파일들은 같은 BIAS, DARK, FLAT 파일을 사용한다. BIAS, DARK, FLAT 파일들은 ``BDFDIR`` 안에 들어 있으며 ``TODODIR`` 안에는 ``BDFDIR`` 푤더는 하만 있어야 합니다. 

아래 코드를 실행하면 ``BDFDIR`` 안의 모든 ``master_files_ys``라는 폴더를 만들어 BIAS, DARK, FLAT의 master file들을 자동으로 합성합니다.

In [ ]:
print("BDFDIR :", BDFDIR)
try :
    _astro_utilities.combine_BDF(BDFDIR,  
                tryagain = tryagain, 
                # file_age = file_age,  #default 365 days 
                verbose = verbose,
                )
except Exception as err :
    print("X"*60)
    print(str(err))
    pass

## 측광을 수행할 폴더

``DOINGDIRs``에 들어 있는 모든 폴더에 대해 측광을 수행한다. 우선 첫번째 폴더만 측광을 수행해 보겠습니다.

In [ ]:
DOINGDIR = DOINGDIRs[0]

## Plate solving

각각의 별들의 좌표를 확인하여 비교 측광을 수행하기 위해서는 plate solving이 필수적이다. plate solving 하는 방법은 다양한데, 다음과 같은 순서대로 시도를 하는 것이 좋습니다.

### ASTAP
가장 빠르고 손쉽게 plate solving을 수행할 수 있는 방법으로 windows, mac, ubuntu 모두 지원하며 자세한 설치 및 사용법은 관련 사이트를 참고하시기 바랍니다.

### astrometry local solver
가장 강력한 plate solving 도구입니다. windows, mac, ubuntu 모두 설치 가능하지만, windows의 경우에는 콘솔 창에서 실행하기 어려워, mac이나 ubuntu 사용을 권장합니다.

### astrometry server
위의 두가지 방법으로 plate solving이 실패할 경우 마지막으로 시도할 방법입니다. api 키를 이용하여 온라인으로 서버에 파일을 전송하여 plate solving을 수행합니다.



> find '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-11_-_GSON300_STF-8300M_-_1bin/XO-6b_LIGHT_-_2025-01-09_-_GSON300_STF-8300M_-_1bin/' -type f -name '*.fit' -exec astap -f '{}' -wcs -analyse2 -update \;
> find '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-11_-_GSON300_STF-8300M_-_1bin/Qatar-9b_LIGHT_-_2025-01-09_-_GSON300_STF-8300M_-_1bin/' -type f -name '*.fit' -exec solve-field -O --downsample 4 --nsigma 30 --cpulimit 20 '{}' \;
> print(f"astap -f {str(fpath)} -fov {hfov} -wcs -analyse2 -update")    
> os.system(f"astap -f {str(fpath)} -fov {hfov} -wcs -analyse2 -update")


In [4]:
try : 
    _astro_utilities.solving_fits_file(DOINGDIR,
            # downsample = downsample, # default 4
            # count_stars = count_stars,
            tryagain = tryagain,
            tryASTAP = True, # default False 
            tryASTROMETRYNET = tryASTROMETRYNET,
            verbose = verbose,
            )  
except Exception as err :
    print("X"*60)
    print(str(err))
    pass

All 49 keywords (guessed from /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/-_CAL-BDF_-_2017-01_-_RiLA600_STX-16803_-_2bin/-_BIAS_-_2017-01-02-22-03-32_0sec_-_STX-16803_-20c_2bin.fit) will be loaded.


## CCD reduction

다음의 코드를 실행하면 CCD preprocessing을 실행합니다.

In [ ]:
try :
    _astro_utilities.ccd_Reduction(DOINGDIR,
            BDFDIR,
            tryagain = False,
            trynightsky = trynightsky,
            # file_age = file_age,  #default 365 days 
            verbose = verbose,
            )
except Exception as err :
    print("X"*60)
    print(str(err))
    pass

## plate solving again

CCD reduction을 수행하기 전에는 plate solving을 실패하지만 이후에 성공하는 사례가 있어 다시한번 plate solving을 시도합니다.


In [ ]:
try :
    _astro_utilities.solving_fits_file(DOINGDIR,
            SOLVINGDIR = _astro_utilities.reduced_dir,
            # downsample = downsample, # default 4
            tryagain = tryagain,
            tryASTAP = True, # default False
            tryASTROMETRYNET = tryASTROMETRYNET,
            verbose = verbose,     
            ) 
except Exception as err :
    print("X"*60)
    print(str(err))
    pass

## Differential photometry using PS1 catalogue

다음 코드를 실행하면 PAN-STARS catalogue를 이용하여 비교 측광을 수행합니다.

In [ ]:
try :
    _astro_utilities.diff_Photometry_PS1(DOINGDIR,
            tryagain = tryagain, # True, 
            LOCATION = LOCATION,
            SKYC_KW = SKYC_KW,
            FWHM_INIT = FWHM_INIT,
            Mag_Low = Mag_Low,
            Mag_High = Mag_High,
            Mag_target = Mag_target,
            Mag_delta = Mag_delta,
            ERR_Max = ERR_Max,
            READINGDIR =  _astro_utilities.reduced_dir,
            # READINGDIR =  _astro_utilities.reduced_nightsky_dir,
            # file_age = file_age,  #default 365 days 
            verbose = verbose,   
            ) 
except Exception as err :
    print("X"*60)
    print(str(err))
    pass
